In [59]:
#@hidden_cell
widgets.HTML('<head><link href="style-def.css" rel="stylesheet" type="text/css"></head><div id="footer"><div class="footerLogoEnergy"><img src="logo_innonet.png" id="logo_energy_router_footer" /></div> <div class="footerCentral">   <div id="div_logo_rep_italiana"><img src="rep_italiana.png" id="logo_it" /> </div><div id="div_ecoloop_description"><p>ECOLOOP è un progetto finanziato nellʼambito del bando INNONETWORK<br />Cod. Progetto: 2AT8246<br />   KET: Micro e Nanoelettronica – Tecnologie per sensori<br />Sfida sociale: Città e territori sostenibili</p> </div><div id="div_logo_reg_puglia"> <img src="reg_puglia.png" id="logo_reg" /></div></div><div class="footerLogoFinc"><img src="logo_fincons.png" id="logo_fincons" /></div></div>')

HTML(value='<head><link href="style-def.css" rel="stylesheet" type="text/css"></head><div id="footer"><div cla…

In [5]:
# @hidden_cell
from ipywidgets import Button, GridBox, Layout, ButtonStyle
from IPython.display import HTML
import ipywidgets as widgets

#Importazione libreria influxdb
from influxdb import InfluxDBClient
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import datetime 
from ipywidgets import Button, HBox, VBox,Layout
import pandas as pd
import ipywidgets as widgets
import iso8601
import matplotlib.pyplot as plt 
from datetime import timezone
import plotly.express as px
from IPython.display import clear_output

#connessione database
client = InfluxDBClient(host='172.25.4.68', port=8086, username='influx_reader', password='inF1uxDB', ssl=False, verify_ssl=False)
#mostrare lista database accessibili
client.get_list_database()
#uso database interessato
client.switch_database('EcoLoopDataField')
#ottengo lista measurement
client.get_list_measurements()



#definizione funzione di ritorno valore tag
def return_label(Osservazione):
    return(Osservazione)
#definizione funzione di selezione del valore tag
def select_sito(Sito): 
    label_dropdown.options = siti[Sito]

#ottengo lista measurement
query_sito =client . query ('SHOW TAG VALUES from "data_campo" WITH KEY = "id_sito"') 
id_sito  =  list ( query_sito . get_points ( ))  
siti={}
#itero sulla lista measurement
for index in range(len(id_sito)):
    #itero sui dizionari di measurement
    for key in id_sito[index]:
        if(key=='value'):
            #assegno a var il nome di ogni measurement
            var =id_sito[index][key]
            query_label  =  client . query ( 'SHOW TAG VALUES from "data_campo" WITH KEY ="label" WHERE "id_sito" =' +'\''+ var +'\'')
          
            #trasformo il resulset in una lista di dizionari
            labels  =  list ( query_label . get_points ( ))
            #ciclo sula lista dei tag per assegnarli alla corrispettiva  measurement
            list_label=[]
            for  i in range(len(labels)):
            #itero sui dizionari per ottenermi il nome della lista dei tag
                for label in labels[i]:
                    if(label=='value'):
                        value =labels[i][label]
                        list_label.append(labels[i][label])
    #assegno a ogni measurement il proprio tag
           
    siti.update({id_sito[index][key]:list_label } )


#creazione  dropdown siti e label
sito_dropdown = widgets.Dropdown(options=siti.keys())
init = sito_dropdown.value
label_dropdown = widgets.Dropdown(options=siti[init])

#creazione iterazione widget
j = widgets.interactive(select_sito,Sito=sito_dropdown)
i= widgets.interactive(return_label, Osservazione=label_dropdown)


# Create buttons for start and end dates
data_inizio = widgets.DatePicker(
    description='Data Inizio',
    disabled=False)
    
ora_inizio = widgets.Dropdown(options=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23 ] 
                             )
minuto_inizio = widgets.Dropdown(options=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60],
                                 description=':',
                                 )
ora_fine = widgets.Dropdown(options=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23 ]
                            )
minuto_fine = widgets.Dropdown(options=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60],
                                description=':'
                               )
                                
# Create buttons for start and end dates
data_fine = widgets.DatePicker(
    description='Data fine ',
    disabled=False,
)

#creazione bottone mostra grafico
button = widgets.Button(description="Mostra grafico")
output = widgets.Output()

#Dispozione widget

osservazione_box=HBox([j,i])
data_inzio_box = HBox([data_inizio,ora_inizio,minuto_inizio])
data_fine_box = HBox([data_fine,ora_fine,minuto_fine])
form=VBox([osservazione_box, data_inzio_box,data_fine_box,button,output])
display(form)



#definizione click bottone   
def on_button_clicked(b):
    with output:
            clear_output()
            #definizione variabili
            list_time=[]
            list_time_to_show=[]
            list_quantity=[]
            list_sensore=[]
            tabella={}
            #Acquisizione valore data inizio e data fine in stringa
            data_inizio_value= str(data_inizio.value)
            data_fine_value= str(data_fine.value)
    
            #acquisizione valori id_sito e label
            label_value= str(label_dropdown.value)
            sito_value= str(sito_dropdown.value)
            
            #cambio formattazione data inizio e data fine 
            data_inizio_valore=  datetime.datetime.strptime(data_inizio_value, "%Y-%m-%d")
            data_fine_valore=  datetime.datetime.strptime(data_fine_value, "%Y-%m-%d")
            
            #acquisizione ora e minuti di inizio e fine 
            ora_inizio_valore= ora_inizio.value
            minuto_inizio_valore= minuto_inizio.value   
            ora_fine_valore= ora_fine.value
            minuto_fine_valore= minuto_fine.value
            
             #creazione timestamp data inizio e fine
            data_inizio_valore=data_inizio_valore.replace(hour=ora_inizio_valore,minute=minuto_inizio_valore,tzinfo=timezone.utc)
            data_fine_valore=data_fine_valore.replace(hour=ora_fine_valore,minute=minuto_fine_valore,tzinfo=timezone.utc)
            timestamp_data_inizio = (round(data_inizio_valore.timestamp()))*1000
            timestamp_data_fine = (round(data_fine_valore.timestamp()))*1000
         
            #definizione query 
            query1='SELECT "quantity","id_sensore" FROM "data_campo" where "time"  >'+ ' '  + str(timestamp_data_inizio) + 'ms' + ' '  +   'AND time <'+ ' '  + str(timestamp_data_fine) + 'ms' + ' ' + 'AND "id_sito"= '+ '\''+ sito_value + '\''+ ' ' + 'AND label='+ ' ' + '\'' +label_value+'\''+ ' '+'GROUP BY "id_sensore"'           
            #esecuzione query
            query=client.query(query1)
            
            #lista risultato query
            list_result = list ( query . get_points ( ))
            #itero sulla lista della query
            for index in range(len(list_result)):
                #itero sui dizionari 
                for (key) in list_result[index]:
                    var =list_result[index]
                                         
                #acquisisco timestamp formato data                         
                time  = datetime.datetime.strptime(var.get('time'), "%Y-%m-%dT%H:%M:%SZ").strftime("%d/%m-%H:%M")
          
                #acquisisco quantità 
                quantity= round (var.get('quantity'),2)
                
                #acquisisco lista sensori
                sensore= var.get('id_sensore')
                
                #aggiungo alle liste i vari dati
                list_time.append(time)
                list_quantity.append(quantity)
                list_sensore.append(sensore)
                
                
                #creazione dizionario di risultati
                tabella.update({sensore:quantity})
            
                      
            
    
            df = pd.DataFrame({"time":list_time,
                   "quantity":list_quantity,
                   "sensore":list_sensore
                    }) 
            # plotting the points
                       

            fig = px.line(df, x="time", y="quantity", color='sensore')
            fig.show()
            
#assegnazione funzione al click
button.on_click(on_button_clicked)

In [4]:
# @hidden_cell
widgets.HTML('<head><link href="style-def.css" rel="stylesheet" type="text/css"></head><div id="footer"><div class="footerLogoEnergy"><img src="logo_innonet.png" id="logo_energy_router_footer" /></div> <div class="footerCentral">   <div id="div_logo_rep_italiana"><img src="rep_italiana.png" id="logo_it" /> </div><div id="div_ecoloop_description"><p>ECOLOOP è un progetto finanziato nellʼambito del bando INNONETWORK<br />Cod. Progetto: 2AT8246<br />   KET: Micro e Nanoelettronica – Tecnologie per sensori<br />Sfida sociale: Città e territori sostenibili</p> </div><div id="div_logo_reg_puglia"> <img src="reg_puglia.png" id="logo_reg" /></div></div><div class="footerLogoFinc"><img src="logo_fincons.png" id="logo_fincons" /></div></div>')

HTML(value='<head><link href="style-def.css" rel="stylesheet" type="text/css"></head><div id="footer"><div cla…